In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from numpy.fft import fft, fftfreq
import matplotlib.pyplot as plt
from scipy import signal 
%matplotlib inline

## To do

- test prediction segment accuracy as a function of segment size (todays task. make an argument about physical significance of 136 or 150)
- Number of NN layers
- Number of layer nodes
- Run predictions using frequency domain info (psd) as opposed to time domain 

## Data preprocessing
Label the data:
Normal: label #1
Inner race fail: label #2
Outer race fail: label #3
Roller defect: label #4

Data includes 3 datasets were created. At the end of the test, the followign was reported:

                  |        |   bearing1   bearing2   bearing3   bearing4|
                  |--------|--------------------------------------------|
                  |        |  Sensor1       sensor2   sensor3   sensor4 |
                  |--------|--------------------------------------------|
                  |Set #1  |    1             1          2          4   |
                  |--------|--------------------------------------------|
                  |Set #2  |    3             1          1          1   | 
                  |--------|--------------------------------------------|  
                  |Set #3  |    1             1          3          1   | 
                  |--------|--------------------------------------------|
                  
Datasets used for model development:

         lable 1: set 2, bearing 2
         label 2: set 1, bearing 3
         label 3: set 3, bearing 3
         label 4: set 1, bearing 4
                  
                  
We assume the above status was valid for that last 20 data collections.
Therefore, we get data from last 20 files, and label them as above.

The form of training and testing data is a pd dataframe:

Columns: set_np, file, seg_no, bearing_no, sensor_outpt, label
Each record is a segment.
data in sensor_outpt are list of 20480/seg for each classification.
seg_no starts is from 1 to seg, for each file
bearing_no is the bearing number that the label applies to.
label indicates the classification of each bearing health from 1 to 4.

In [2]:
def get_xy(file, set_no, bearing_no, label, domain='time'):
    ''' Return the X (features) and y (label) for model prediction
    file: the file name
    **src is a dic and includes set_no, bearing_no and label
    domain corresponds to X and can be time or freq. 
        
    '''
    file_path = get_path(set_no, file)
    f = pd.read_csv(file_path, delimiter='\t', header=None)
    seg_size = int(f.shape[0] / seg_num)
    X_cols = [i for i in range(seg_size)]
    # Trim the data in set#1 to make one sensor per bearing
    if set_no == '1':
        f.drop(f.columns[[1, 3, 5, 7]], axis=1, inplace=True)  
    X = pd.DataFrame(columns=X_cols)
    y = pd.Series()
    for seg_no in range(seg_num):
        min_indx = seg_no * seg_size
        max_indx = (seg_no + 1) * seg_size 
        col_num = bearing_no - 1
        if domain == 'time':
            X_temp = f.iloc[min_indx:max_indx, col_num]
            X_temp = pd.Series(X_temp)
            X_temp.index = X_cols
        # To convert X into freq domain:
        else:
            X_temp = f.iloc[min_indx:max_indx, col_num]
            fft_vals = fft(X_temp)
            n = X_temp.size
            psd = 2*np.abs(fft_vals/n)**2
            X_temp = pd.Series(psd)
            #X_temp.index = 
        X = X.append(X_temp, ignore_index=True)
        y = y.append(pd.Series([label]))
    return X, y   

In [3]:
def get_path(set_no, file):
    ''' Return full path to the file
    set_no is the dataset number (1,2, or 3)
    file is the file name (e.g. '2003.10.22.12.39.13')
    
    '''
    path = os.path.join(datasets_dir, set_no, file)
    return path
    

In [4]:
def get_files(set_no, files_num=0):
    ''' Return the names of the last file_num files in the dataset set_no
        if files_num is 0, then it returns all files in the directory
        otherwise, it returned files_num number of files
    '''
    file_dir = os.path.join(datasets_dir, set_no)
    all_files = os.listdir(file_dir)
    if files_num == 0:
        files_num = len(all_files)
    all_files.sort(reverse=True)
    latest_files = all_files[0:files_num]
    return latest_files

In [5]:
def seg_accuracy(y_pred, y_test):
    ''' Return 4 measures of accuracy based on analysis of a single segment.
    Returns: [accuracy of correct prediction of label1, ..., accuracy of correct prediction of label 4]
        
    '''
    ac=[]
    for label in range(1, 5):
        y_pred_bool= y_pred == str(label)
        y_test_bool= y_test == str(label)
        accuracy = (y_pred_bool == y_test_bool).mean()
        ac.append(accuracy)
    return ac

In [6]:
def get_accuracy(y):
    ''' Return accuracy of the model (from 0 to 1).
    y is a list of (y_prediction, label). Each element of the list corresponds to one file
    
    '''
    correct_pred = [int(pred) == label for pred, label in pred_lst]
    accuracy = sum (correct_pred)/len(pred_lst)
    return accuracy

In [7]:
def get_label(set_no, bearing_no):
    ''' Finds label (1, 2, 3 or 4) for each of the four bearings, based 
    on the dataset number.
    Return the element in the label list that corresponds to the bearing of interest
    
    '''
    if set_no == '1':
        label=[1, 1, 2, 4]
    if set_no == '2':
        label=[3, 1, 1, 1]  
    if set_no == '3':
        label=[1, 1, 3, 1]
    res = label[bearing_no - 1]
    return res
    

In [8]:
def overal_pred(X, clf):
    '''Calculate the overal probability for each class, and return the class with max probability
    
    '''
    prob_seg = clf.predict_proba(X) 
    prob_overal = prob_seg.sum(axis=0)
    max_prob = max(prob_overal)
    maxindx = prob_overal.argmax()
    classification = maxindx + 1
    return classification
    

In [9]:
def pred_all_sets(domain):
    ''' Return the prediction for all files_no files in all sets
    retuns a list of (prediction, actual label) for every files used.
    domain can be time or freq
    '''
    pred_lst=[]
    files_no = 20
    for set_no in ['1', '2', '3']:
        files = get_files(set_no, files_no)
        for file in files:
            for bearing_no in [1, 2, 3, 4]:
                label = get_label(set_no, bearing_no)
                src = src = {'set_no': set_no, 'bearing_no': bearing_no, 'label': label}
                X, y = get_xy(file, **src, domain=domain)
                pred = overal_pred(X, clf)
                pred_lst.append((pred, label))
    return pred_lst

### Set parameters

In [10]:
datasets_dir = 'C:\\Users\\10134838\\Desktop\\PProjects\\Sensors\\Bearing_Cincinati\\'
seg_num = 136
feature_size = 20480 // seg_num
X = pd.DataFrame()
y = pd.Series()
# Number of files from each dataset for segmentation (e.g. files_num = 2 means the last two files)

## Train model

In [11]:
def train_test(domain, n_hidden_layers, training_file_no):
    ''' Specify these four groups as sources of data
             lable 1: set 2, bearing 2
             label 2: set 1, bearing 3
             label 3: set 3, bearing 3
             label 4: set 1, bearing 4
             domain can be time or freq
    '''
    # files_num is the number of files used in each set to train model
    src1 = {'set_no': '2', 'bearing_no': 2, 'label': '1'}
    src2 = {'set_no': '1', 'bearing_no': 3, 'label': '2'}
    src3 = {'set_no': '3', 'bearing_no': 3, 'label': '3'}
    src4 = {'set_no': '1', 'bearing_no': 4, 'label': '4'}
    src_all = [src1, src2, src3, src4]
    # Get the dataframe of data for testing and training
    X = pd.DataFrame()
    y = pd.Series()
    for src in src_all:
        files = get_files(src['set_no'], training_file_no)
        for file in files:
            X_temp, y_temp = get_xy(file, **src, domain=domain)
            X = X.append(X_temp)
            y = y.append(y_temp)


    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=1)
    clf = MLPClassifier(hidden_layer_sizes=n_hidden_layers)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    seg_ac = sum(seg_accuracy(y_pred, y_test))/4
    return seg_ac, clf

##  Predictions
We will investigate two way of testing the model:
first, test on segments that have not been used for training. 
In simplest form, prediction is based on a single segment. 
For now, lets stick to prediction based on single segment

Above predictions are for each segments. Below, we get the overal prediction:

Overal propability = sum of all segment probabilities
Overal prediction corresponds to the class with maximum overal probability

    

In [12]:
domains = ['time', 'freq']
training_file_nos = [10, 20]
seg_nos = [136]
ns_hidden_layers = [(100,),(100, 100), (100,50), (100,50,25), (100,50, 25, 12), (100,100, 50, 25, 12)]
pred_df_cols = ['training_file_no', 'seg_no', 'domain', 'n_hidden_layers', 'seg_ac', 'overal_ac']
# pred_df is a df containing on the predictions and the parameters
pred_df = pd.DataFrame(columns=pred_df_cols)
for training_file_no in training_file_nos:
    for seg_no in seg_nos:
        for domain in domains:
            for n_hidden_layers in ns_hidden_layers:
                print(n_hidden_layers)
                seg_ac, clf = train_test(domain, n_hidden_layers, training_file_no)
                pred_lst = pred_all_sets(domain)
                overal_ac = get_accuracy(pred_lst)
                record = pd.Series([training_file_no, seg_no, 
                                    domain, n_hidden_layers, seg_ac, overal_ac], index=pred_df_cols)
                pred_df = pred_df.append(record, ignore_index=True)

(100,)
(100, 100)
(100, 50)
(100, 50, 25)
(100, 50, 25, 12)
(100, 100, 50, 25, 12)
(100,)


C:\Users\10134838\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


(100, 100)
(100, 50)
(100, 50, 25)
(100, 50, 25, 12)
(100, 100, 50, 25, 12)
(100,)
(100, 100)
(100, 50)
(100, 50, 25)
(100, 50, 25, 12)
(100, 100, 50, 25, 12)
(100,)
(100, 100)
(100, 50)
(100, 50, 25)
(100, 50, 25, 12)
(100, 100, 50, 25, 12)


Conclusions:
    - Segment accuracy good indicator of overal accuracy
    - Best results was overal prediction of 82% which was achieved in frequency domain with 2 hidden layer NN (100,50)
    - All errors were false positive, which is more acceptable to industry
    - To improve results, add more sensors or train each sensor seperately.